## Imports

In [2]:
import os
import dlib
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
import scipy
import pandas as pd
import random
import math
from sklearn import svm

## Preprocessing:

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../shape_predictor_68_face_landmarks.dat")

In [7]:
def write_cropped_image(orig_path, new_path=None):
    print("Processing file: {}".format(orig_path))
    img = dlib.load_rgb_image(orig_path)
    face_area = detector(img, 1)
    shape = predictor(img, face_area[0])
    pts = np.array( # the 27 points on the face
        [[shape.part(0).x, shape.part(0).y], [shape.part(3).x, shape.part(3).y],
         [shape.part(4).x, shape.part(4).y], [shape.part(5).x, shape.part(5).y], 
         [shape.part(6).x, shape.part(6).y], [shape.part(7).x, shape.part(7).y],
         [shape.part(8).x, shape.part(8).y], [shape.part(9).x, shape.part(9).y],
         [shape.part(10).x, shape.part(10).y], [shape.part(11).x, shape.part(11).y], 
         [shape.part(12).x, shape.part(12).y], [shape.part(13).x, shape.part(13).y],
         [shape.part(14).x, shape.part(14).y], [shape.part(15).x, shape.part(15).y],
         [shape.part(16).x, shape.part(16).y], [shape.part(26).x, shape.part(26).y], 
         [shape.part(25).x, shape.part(25).y], [shape.part(24).x, shape.part(24).y],
         [shape.part(23).x, shape.part(23).y], [shape.part(22).x, shape.part(22).y],
         [shape.part(27).x, shape.part(27).y], [shape.part(21).x, shape.part(21).y], 
         [shape.part(20).x, shape.part(20).y], [shape.part(19).x, shape.part(19).y],
         [shape.part(18).x, shape.part(18).y], [shape.part(17).x, shape.part(17).y]], 
         dtype=np.int32)
    # Create polygon shaped mask
    mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.int32)
    cv2.fillPoly(mask, np.int32([pts]), 1)
    mask = mask.astype(bool)
    # Fill in polygon with image
    out = np.zeros_like(img)
    out[mask] = img[mask]

    #src = cv2.imread(f, 0)
    cropped_image = out[int(face_area[0].top()):int(face_area[0].bottom()), int(face_area[0].left()):int(face_area[0].right())]
    cropped_image = cv2.resize(cropped_image, (130, 130))
    
    #plt.imshow(cropped_image)
    #plt.show()
    cv2.imwrite(new_path, cropped_image)


In [ ]:
write_cropped_image("../Ck+/anger/S503_001_00000071.png", "a")

In [ ]:
# Given a folder
def preprocess_folder_of_emotions(folder_path, emotions_list, pic_type="*.png"):
    write_to = folder_path + "_new"
    for emotion in emotions_list:
        #print("first for loop")
        #print(pic_type)
        emo_folder_path = folder_path + "/" + emotion
        write_path = write_to + "/" + emotion
        #print("elp_0")
        for pic_file in glob.glob(os.path.join(emo_folder_path, pic_type)):
            #print("elp")
            #print("Writing file: {}".format(pic_file))
            write_pic_path = write_path + pic_file[-22:]
            write_cropped_image(pic_file, write_pic_path)

                                  
emotions_list = ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]
preprocess_folder_of_emotions("../CK+", emotions_list, "*.png")

## LBP Feature Vectors

In [4]:
# LBP
emotions_list = ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]
list_of_vecs = []
counter = 0
for a in emotions_list:
    faces_folder_path = "../CK+_new/" + str(a)
    for pic_file in glob.glob(os.path.join(faces_folder_path, "*.png")):
        src = cv2.imread(pic_file, 0)
        lbp_image = local_binary_pattern(src, 8, 2, method='nri_uniform')
        histogram = scipy.stats.itemfreq(lbp_image)
        vector = []
        for c, b in histogram:
            vector.append(b)
        
        list_of_vecs.append([counter, a, vector])
        counter +=1
print(len(list_of_vecs[0][2]))

/var/folders/d6/73t68jg509l9qqjp_k_6st940000gp/T/ipykernel_72974/1619009774.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  histogram = scipy.stats.itemfreq(lbp_image)


59


In [5]:
print(list_of_vecs[:2])

[[0, 'anger', [272.0, 179.0, 11.0, 50.0, 29.0, 199.0, 21.0, 66.0, 16.0, 125.0, 77.0, 38.0, 59.0, 97.0, 103.0, 36.0, 47.0, 286.0, 165.0, 173.0, 165.0, 213.0, 128.0, 231.0, 224.0, 698.0, 329.0, 267.0, 625.0, 596.0, 230.0, 209.0, 882.0, 338.0, 208.0, 216.0, 142.0, 283.0, 163.0, 141.0, 185.0, 61.0, 85.0, 115.0, 104.0, 42.0, 39.0, 89.0, 95.0, 39.0, 23.0, 59.0, 133.0, 31.0, 21.0, 42.0, 122.0, 6375.0, 903.0]], [1, 'anger', [170.0, 148.0, 9.0, 58.0, 13.0, 164.0, 10.0, 35.0, 17.0, 73.0, 49.0, 39.0, 30.0, 35.0, 76.0, 38.0, 35.0, 233.0, 65.0, 100.0, 88.0, 108.0, 27.0, 170.0, 117.0, 519.0, 517.0, 67.0, 360.0, 244.0, 98.0, 180.0, 468.0, 329.0, 158.0, 182.0, 77.0, 296.0, 70.0, 215.0, 117.0, 54.0, 57.0, 131.0, 80.0, 55.0, 60.0, 69.0, 104.0, 109.0, 26.0, 149.0, 200.0, 119.0, 31.0, 112.0, 142.0, 8089.0, 1509.0]]]


## ORB Vectors

In [6]:
counter = 0
for a in emotions_list:
    faces_folder_path = "../CK+_new/" + a
    for f in glob.glob(os.path.join(faces_folder_path, "*.png")):
        #print("Processing file: {}".format(f))
        src = cv2.imread(f, 0)
        src = src[1:129, 1:129]
        #print(src.shape)
        #print(len(src[0]))
        #print('hi')
        # Initiate ORB detector
        orb = cv2.ORB_create()
        # find the keypoints with ORB
        kp = orb.detect(src, None)
        # compute the descriptors with ORB
        kp, des = orb.compute(src, kp)
        #print(des[0])
        #print(des[0].shape)
        #print(des[0].tolist())
        list_of_vecs[counter].append(des[0].tolist())
        counter += 1

In [8]:
print(list_of_vecs[0:2])

[[0, 'anger', [272.0, 179.0, 11.0, 50.0, 29.0, 199.0, 21.0, 66.0, 16.0, 125.0, 77.0, 38.0, 59.0, 97.0, 103.0, 36.0, 47.0, 286.0, 165.0, 173.0, 165.0, 213.0, 128.0, 231.0, 224.0, 698.0, 329.0, 267.0, 625.0, 596.0, 230.0, 209.0, 882.0, 338.0, 208.0, 216.0, 142.0, 283.0, 163.0, 141.0, 185.0, 61.0, 85.0, 115.0, 104.0, 42.0, 39.0, 89.0, 95.0, 39.0, 23.0, 59.0, 133.0, 31.0, 21.0, 42.0, 122.0, 6375.0, 903.0], [59, 34, 68, 1, 67, 90, 96, 64, 117, 100, 230, 171, 75, 12, 150, 212, 120, 140, 234, 132, 13, 178, 229, 128, 22, 96, 128, 3, 103, 140, 94, 196]], [1, 'anger', [170.0, 148.0, 9.0, 58.0, 13.0, 164.0, 10.0, 35.0, 17.0, 73.0, 49.0, 39.0, 30.0, 35.0, 76.0, 38.0, 35.0, 233.0, 65.0, 100.0, 88.0, 108.0, 27.0, 170.0, 117.0, 519.0, 517.0, 67.0, 360.0, 244.0, 98.0, 180.0, 468.0, 329.0, 158.0, 182.0, 77.0, 296.0, 70.0, 215.0, 117.0, 54.0, 57.0, 131.0, 80.0, 55.0, 60.0, 69.0, 104.0, 109.0, 26.0, 149.0, 200.0, 119.0, 31.0, 112.0, 142.0, 8089.0, 1509.0], [11, 158, 246, 223, 175, 254, 188, 151, 191, 223

## Post-Processing

In [44]:
def z_fold_thingy(full_list):
    overall_list = []
    for val in full_list:
        vector_LBP = val[2]
        vector_ORB = val[3]
        max_lbp = max(vector_LBP)
        vector_LBP = [a/max_lbp for a in vector_LBP]

        max_orb = max(vector_ORB)
        vector_ORB = [a/max_orb for a in vector_ORB]
        overall_list.append([val[1], vector_LBP+vector_ORB])

    c = 1e-5
    for counter, (emo, vector) in enumerate(overall_list):
        #print(emo)
        #print(vector)
        avg = sum(vector)/len(vector)
        avg2 = 0
        r = 0
        
        for a in vector:
            avg2 +=a/91
        for a in vector:
            r += (a-avg)**2
        
        new = []
        for counter2, a in enumerate(vector):
            #if counter < 1:
            #    print(a)
            #    print(val)
            val = 100 * (a-avg)/(r+c)
            
            new.append(val)
        #print(len(overall_list[counter][1]))
        #print(len(new))
        overall_list[counter][1] = new
    return overall_list

In [45]:
print(z_fold_thingy(list_of_vecs)[0])

['anger', [-2.0754366074346406, -2.276046620809393, -2.63843890303475, -2.554312123232435, -2.5996111585106045, -2.232904682449231, -2.6168679338546688, -2.5197985725443055, -2.6276534184447096, -2.392529854381829, -2.4960705064462165, -2.580197286248532, -2.534898250970362, -2.4529285680860546, -2.439985986578007, -2.5845114800845477, -2.560783413986459, -2.0452372505825274, -2.3062459776615056, -2.2889892023174405, -2.3062459776615056, -2.2027053255971176, -2.3860585636278047, -2.1638775810729722, -2.1789772594990287, -1.1565133203631979, -1.95248208310818, -2.086222092024681, -1.3139813953777881, -1.3765372060000225, -2.16603467799098, -2.21133371326915, -0.7596074874497108, -1.9330682108461075, -2.213490810187158, -2.1962340348430933, -2.3558592067756914, -2.0517085413365517, -2.3105601714975212, -2.3580163036936996, -2.2631040393013437, -2.530584057134346, -2.4788137311021523, -2.4141008235619092, -2.437828889659998, -2.5715688985765, -2.5780401893305234, -2.4701853434301198, -2.4

In [56]:
print(random.sample(range(len(list_of_vecs)), 5))
    

[571, 466, 210, 625, 848]


In [59]:
def train_svm_2(train_percent, trials, list_of_vecs):
    emo_dict = {"anger": 0, "contempt": 1, "disgust": 2, "fear": 3, "happiness": 4, "neutral": 5, "sadness": 6,
            "surprise": 7}
    #print(list_of_vecs)
    shuffle_desk = [a for a in list_of_vecs]
    
    #print("1")
    list_of_emotion_of_vectors = {} # Creates a dictionary
    #print(shuffle_desk)
    for [emo,vector] in shuffle_desk:
        #print(emo)
        if emo not in list_of_emotion_of_vectors.keys():
            list_of_emotion_of_vectors[emo] = [vector]
        else:
            list_of_emotion_of_vectors[emo].append(vector)
          
    #print(shuffle_desk[0])
    #print(len(shuffle_desk))
    #train_count = int((len(list_of_vecs) * (train_percent/100))//1) 
    #print(train_count)
    #print("before for loop")
    for a in range(trials):
        train_vectors = []
        train_labels = []
        test_vectors = []
        test_labels = []
        #print(list_of_emotion_of_vectors)
        for emotion in list_of_emotion_of_vectors.keys():
            #print("Keys:")
            #print(emotion)
            length = len(list_of_emotion_of_vectors[emotion])
            chosen = int(math.ceil(length * (train_percent/100))) # round down
            #print("chosen")
            #print(chosen)
            #print(length)
            if length > 100:
                for vector in list_of_emotion_of_vectors[emotion][:train_percent]:
                    #print("train")
                    train_vectors.append(vector)
                    train_labels.append(emo_dict[emotion])
                for vector in list_of_emotion_of_vectors[emotion][train_percent:100]:
                    #print("test")
                    #print(emo)
                    test_vectors.append(vector)
                    test_labels.append(emo_dict[emotion])
            else:
                for vector in list_of_emotion_of_vectors[emotion][:chosen]:
                    #print("train")
                    train_vectors.append(vector)
                    train_labels.append(emo_dict[emotion])
                for vector in list_of_emotion_of_vectors[emotion][chosen:]:
                    #print("test")
                    #print(emo)
                    test_vectors.append(vector)
                    test_labels.append(emo_dict[emotion])
        #print(test_labels)
    
         
        # Train the SVM
#        svm_object = svm.SVC(kernel='rbf', gamma=100, C=1e-6) # 64 with normal set

        svm_object = svm.SVC(kernel='rbf', gamma="auto")        

        print(test_labels)
        svm_object.fit(np.asarray(train_vectors), train_labels)

        
        # Test it
        # Confusion matrix: column headers = correct, rows = what was outputted
        confusion_matrix_count = np.zeros((8,8))
        right = 0
        wrong = 0
        #print(len(test_labels))
        for label, vector in zip(test_labels, np.asarray(test_vectors)):
            response = svm_object.predict([vector])
            confusion_matrix_count[int(response)][label] += 1
            #print(response)
            if response == label:
                right += 1
            else:
                wrong += 1
        print(right/(right + wrong) * 100)
        print(confusion_matrix_count)
    

In [60]:
thingy = z_fold_thingy(list_of_vecs)
train_svm_2(95, 1, thingy)

# With train data:
#83.48519362186788
#[[ 15.   0.   0.   0.   0.   0.   0.   0.]
# [  0.   3.   0.   0.   0.   0.   0.   0.]
# [  0.   0.  35.   0.   0.   0.   0.   0.]
# [  0.   0.   0.  11.   0.   0.   0.   0.]
# [  0.   0.   0.   0.  36.   0.   0.   0.]
# [ 28.  15.  22.  13.  30. 564.  19.  18.]
# [  0.   0.   0.   0.   0.   0.   8.   0.]
# [  0.   0.   0.   0.   0.   0.   0.  61.]]

# With test data: 69.04761904761905
#[[ 0.  0.  0.  0.  0.  0.  0.  0.]
# [ 0.  0.  0.  0.  0.  0.  0.  0.]
# [ 0.  0.  0.  0.  0.  0.  0.  0.]
# [ 0.  0.  0.  0.  0.  0.  0.  0.]
# [ 0.  0.  0.  0.  0.  0.  0.  0.]
# [ 2.  0.  2.  1.  3. 29.  1.  4.]
# [ 0.  0.  0.  0.  0.  0.  0.  0.]
# [ 0.  0.  0.  0.  0.  0.  0.  0.]]

[0, 0, 2, 2, 3, 4, 4, 4, 5, 5, 5, 5, 5, 6, 7, 7, 7, 7]
27.77777777777778
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 2. 3. 1. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 2. 0. 2.]]


In [115]:
def train_svm_3(shift, trials, list_of_vecs):
    emo_dict = {"anger": 0, "contempt": 1, "disgust": 2, "fear": 3, "happiness": 4, "neutral": 5, "sadness": 6,
            "surprise": 7}
    #print(list_of_vecs)
    shuffled_desk = [a for a in list_of_vecs]
    label_emotions = []
    data_vector_list = []
    test_emo = []
    test_vector = []

    for a in range(920):
        print("hi")
        # exclude 5% of each?
        # emo_dict_count = {'anger': 45, 'contempt': 18, 'disgust': 59, 'fear': 25, 'happiness': 69, 'neutral': 593,
        # 'sadness': 28, 'surprise': 83}
        # Maybe used F-1 score
        # Something which accounts for the imbalanced counts on each thing
        if a in list(range(-shift, 920, 20)):
            emotion, vector = shuffled_desk[a]
            test_emo.append(emo_dict[emotion])
            test_vector.append(vector)
            continue
        emotion, vector = shuffled_desk[a]
        data_vector_list.append(vector)
        label_emotions.append(emo_dict[emotion])  
    
        labels = np.asarray(label_emotions, dtype=int)
        trainingData = np.asarray(data_vector_list, dtype=np.float32)

        testEmo = np.asarray(test_emo, dtype=int)
        testData = np.asarray(test_vector, dtype=np.float32)
    
        # Train the SVM
        svm_object = svm.NuSVC(kernel="rbf", gamma="auto")
        print(labels)
        svm_object.fit(trainingData, labels)
        
        # Test it
        # Confusion matrix: column headers = correct, rows = what was outputted
        confusion_matrix_count = np.zeros((8,8))
        right = 0
        wrong = 0
        #print(len(test_labels))
        for label, vector in zip(testEmo, testData):
            response = svm_object.predict([vector])
            confusion_matrix_count[int(response)][label] += 1
            print(response)
            if response == label:
                right += 1
            else:
                wrong += 1
        print(right/(right + wrong) * 100)
        print(confusion_matrix_count)
    

In [116]:
thingy = z_fold_thingy(list_of_vecs)
train_svm_3(0, 1, thingy)

hi
hi
[0]


ValueError: The number of classes has to be greater than one; got 1 class